In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


<a id='working-with-missing-data'></a>

# Working with missing data

In this section, we will discuss missing (also referred to as NA) values in
pandas.

>**Note**
>
>The choice of using `NaN` internally to denote missing data was largely
for simplicity and performance reasons.
Starting from pandas 1.0, some optional data types start experimenting
with a native `NA` scalar using a mask-based approach. See
here for more.

See the [cookbook](44_cookbook.ipynb#cookbook-missing-data) for some advanced strategies.

## Values considered “missing”

As data comes in many shapes and forms, pandas aims to be flexible with regard
to handling missing data. While `NaN` is the default missing value marker for
reasons of computational speed and convenience, we need to be able to easily
detect this value with data of different types: floating point, integer,
boolean, and general object. In many cases, however, the Python `None` will
arise and we wish to also consider that “missing” or “not available” or “NA”.

>**Note**
>
>If you want to consider `inf` and `-inf` to be “NA” in computations,
you can set `pandas.options.mode.use_inf_as_na = True`.


<a id='missing-isna'></a>

In [ ]:
df = pd.DataFrame(
    np.random.randn(5, 3),
    index=["a", "c", "e", "f", "h"],
    columns=["one", "two", "three"],
)
df["four"] = "bar"
df["five"] = df["one"] > 0
df

In [ ]:
df2 = df.reindex(["a", "b", "c", "d", "e", "f", "g", "h"])
df2

To make detecting missing values easier (and across different array dtypes),
pandas provides the `isna()` and
`notna()` functions, which are also methods on
Series and DataFrame objects:

In [ ]:
df2["one"]

In [ ]:
pd.isna(df2["one"])

In [ ]:
df2["four"].notna()

In [ ]:
df2.isna()

warning:

> One has to be mindful that in Python (and NumPy), the `nan's` don’t compare equal, but `None's` **do**.
Note that pandas/NumPy uses the fact that `np.nan != np.nan`, and treats `None` like `np.nan`.

In [ ]:
None == None  # noqa: E711

In [ ]:
np.nan == np.nan

So as compared to above, a scalar equality comparison versus a `None/np.nan` doesn’t provide useful information.

In [ ]:
df2["one"] == np.nan


### Integer dtypes and missing data

Because `NaN` is a float, a column of integers with even one missing values
is cast to floating-point dtype (see [Support for integer NA](43_FAQ_gotchas.ipynb#gotchas-intna) for more). pandas
provides a nullable integer array, which can be used by explicitly requesting
the dtype:

In [ ]:
pd.Series([1, 2, np.nan, 4], dtype=pd.Int64Dtype())

Alternatively, the string alias `dtype='Int64'` (note the capital `"I"`) can be
used.

See [Nullable integer data type](31_Nullable_integer_na.ipynb#integer-na) for more.

### Datetimes

For datetime64[ns] types, `NaT` represents missing values. This is a pseudo-native
sentinel value that can be represented by NumPy in a singular dtype (datetime64[ns]).
pandas objects provide compatibility between `NaT` and `NaN`.

In [ ]:
df2 = df.copy()
df2["timestamp"] = pd.Timestamp("20120101")

In [ ]:
df2

In [ ]:
df2.loc[["a", "c", "h"], ["one", "timestamp"]] = np.nan

In [ ]:
df2

In [ ]:
df2.dtypes.value_counts()


<a id='missing-inserting'></a>

## Inserting missing data

You can insert missing values by simply assigning to containers. The
actual missing value used will be chosen based on the dtype.

For example, numeric containers will always use `NaN` regardless of
the missing value type chosen:

In [ ]:
s = pd.Series([1, 2, 3])
s.loc[0] = None
s

Likewise, datetime containers will always use `NaT`.

For object containers, pandas will use the value given:

In [ ]:
s = pd.Series(["a", "b", "c"])
s.loc[0] = None
s.loc[1] = np.nan
s


<a id='working-with-missing-data-calculations'></a>

## Calculations with missing data

Missing values propagate naturally through arithmetic operations between pandas
objects.

In [ ]:
df = df2.loc[:, ["one", "two", "three"]]
a = df2.loc[df2.index[:5], ["one", "two"]].fillna(method="pad")
b = df2.loc[df2.index[:5], ["one", "two", "three"]]

In [ ]:
a

In [ ]:
b

In [ ]:
a + b

The descriptive statistics and computational methods discussed in the
data structure overview (and listed here and [here](../05_reference/13_dataFrame.ipynb#api-dataframe-stats)) are all written to
account for missing data. For example:

- When summing data, NA (missing) values will be treated as zero.  
- If the data are all NA, the result will be 0.  
- Cumulative methods like `cumsum()` and `cumprod()` ignore NA values by default, but preserve them in the resulting arrays. To override this behaviour and include NA values, use `skipna=False`.  

In [ ]:
df

In [ ]:
df["one"].sum()

In [ ]:
df.mean(1)

In [ ]:
df.cumsum()

In [ ]:
df.cumsum(skipna=False)


<a id='working-with-missing-data-numeric-sum'></a>

## Sum/prod of empties/nans

warning:

> This behavior is now standard as of v0.22.0 and is consistent with the default in `numpy`; previously sum/prod of all-NA or empty Series/DataFrames would return NaN.
See [v0.22.0 whatsnew](../03_whatsnew/v0.22.0.ipynb#whatsnew-0220) for more.


The sum of an empty or all-NA Series or column of a DataFrame is 0.

In [ ]:
pd.Series([np.nan]).sum()

In [ ]:
pd.Series([], dtype="float64").sum()

The product of an empty or all-NA Series or column of a DataFrame is 1.

In [ ]:
pd.Series([np.nan]).prod()

In [ ]:
pd.Series([], dtype="float64").prod()

## NA values in GroupBy

NA groups in GroupBy are automatically excluded. This behavior is consistent
with R, for example:

In [ ]:
df

In [ ]:
df.groupby("one").mean()

See the groupby section [here](36_groupby.ipynb#groupby-missing) for more information.

### Cleaning / filling missing data

pandas objects are equipped with various data manipulation methods for dealing
with missing data.


<a id='working-with-missing-data-fillna'></a>

## Filling missing values: fillna

`fillna()` can “fill in” NA values with non-NA data in a couple
of ways, which we illustrate:

**Replace NA with a scalar value**

In [ ]:
df2

In [ ]:
df2.fillna(0)

In [ ]:
df2["one"].fillna("missing")

**Fill gaps forward or backward**

Using the same filling arguments as reindexing, we
can propagate non-NA values forward or backward:

In [ ]:
df

In [ ]:
df.fillna(method="pad")


<a id='working-with-missing-data-fillna-limit'></a>
**Limit the amount of filling**

If we only want consecutive gaps filled up to a certain number of data points,
we can use the `limit` keyword:

In [ ]:
df.iloc[2:4, :] = np.nan

In [ ]:
df

In [ ]:
df.fillna(method="pad", limit=1)

To remind you, these are the available filling methods:

|Method|Action|
|:----------------------------|:------------------------------------------------|
|pad / ffill|Fill values forward|
|bfill / backfill|Fill values backward|
With time series data, using pad/ffill is extremely common so that the “last
known value” is available at every time point.

`ffill()` is equivalent to `fillna(method='ffill')`
and `bfill()` is equivalent to `fillna(method='bfill')`


<a id='working-with-missing-data-pandasobject'></a>

## Filling with a PandasObject

You can also fillna using a dict or Series that is alignable. The labels of the dict or index of the Series
must match the columns of the frame you wish to fill. The
use case of this is to fill a DataFrame with the mean of that column.

In [ ]:
dff = pd.DataFrame(np.random.randn(10, 3), columns=list("ABC"))
dff.iloc[3:5, 0] = np.nan
dff.iloc[4:6, 1] = np.nan
dff.iloc[5:8, 2] = np.nan
dff

In [ ]:
dff.fillna(dff.mean())

In [ ]:
dff.fillna(dff.mean()["B":"C"])

Same result as above, but is aligning the ‘fill’ value which is
a Series in this case.

In [ ]:
dff.where(pd.notna(dff), dff.mean(), axis="columns")


<a id='working-with-missing-data-dropna'></a>

## Dropping axis labels with missing data: dropna

You may wish to simply exclude labels from a data set which refer to missing
data. To do this, use `dropna()`:

In [ ]:
df["two"] = df["two"].fillna(0)
df["three"] = df["three"].fillna(0)
df

In [ ]:
df.dropna(axis=0)

In [ ]:
df.dropna(axis=1)

In [ ]:
df["one"].dropna()

An equivalent `dropna()` is available for Series.
DataFrame.dropna has considerably more options than Series.dropna, which can be
examined [in the API](../05_reference/13_dataFrame.ipynb#api-dataframe-missing).


<a id='working-with-missing-data-interpolate'></a>

## Interpolation

Both Series and DataFrame objects have `interpolate()`
that, by default, performs linear interpolation at missing data points.

In [ ]:
np.random.seed(123456)
idx = pd.date_range("1/1/2000", periods=100, freq="BM")
ts = pd.Series(np.random.randn(100), index=idx)
ts[1:5] = np.nan
ts[20:30] = np.nan
ts[60:80] = np.nan
ts = ts.cumsum()

In [ ]:
ts

In [ ]:
ts.count()

In [ ]:
ts.plot()

In [ ]:
ts.interpolate()

In [ ]:
ts.interpolate().count()

In [ ]:
ts.interpolate().plot()

Index aware interpolation is available via the `method` keyword:

In [ ]:
ts2 = ts[[0, 1, 30, 60, 99]]

In [ ]:
ts2

In [ ]:
ts2.interpolate()

In [ ]:
ts2.interpolate(method="time")

For a floating-point index, use `method='values'`:

In [ ]:
idx = [0.0, 1.0, 10.0]
ser = pd.Series([0.0, np.nan, 10.0], idx)

In [ ]:
ser

In [ ]:
ser.interpolate()

In [ ]:
ser.interpolate(method="values")

You can also interpolate with a DataFrame:

In [ ]:
df = pd.DataFrame(
    {
        "A": [1, 2.1, np.nan, 4.7, 5.6, 6.8],
        "B": [0.25, np.nan, np.nan, 4, 12.2, 14.4],
    }
)
df

In [ ]:
df.interpolate()

The `method` argument gives access to fancier interpolation methods.
If you have [scipy](https://www.scipy.org) installed, you can pass the name of a 1-d interpolation routine to `method`.
You’ll want to consult the full scipy interpolation [documentation](https://docs.scipy.org/doc/scipy/reference/interpolate.html#univariate-interpolation) and reference [guide](https://docs.scipy.org/doc/scipy/reference/tutorial/interpolate.html) for details.
The appropriate interpolation method will depend on the type of data you are working with.

- If you are dealing with a time series that is growing at an increasing rate,
  `method='quadratic'` may be appropriate.  
- If you have values approximating a cumulative distribution function,
  then `method='pchip'` should work well.  
- To fill missing values with goal of smooth plotting, consider `method='akima'`.  


warning:

> These methods require `scipy`.

In [ ]:
df.interpolate(method="barycentric")

In [ ]:
df.interpolate(method="pchip")

In [ ]:
df.interpolate(method="akima")

When interpolating via a polynomial or spline approximation, you must also specify
the degree or order of the approximation:

In [ ]:
df.interpolate(method="spline", order=2)

In [ ]:
df.interpolate(method="polynomial", order=2)

Compare several methods:

In [ ]:
np.random.seed(2)

ser = pd.Series(np.arange(1, 10.1, 0.25) ** 2 + np.random.randn(37))
missing = np.array([4, 13, 14, 15, 16, 17, 18, 20, 29])
ser[missing] = np.nan
methods = ["linear", "quadratic", "cubic"]

df = pd.DataFrame({m: ser.interpolate(method=m) for m in methods})
df.plot()

Another use case is interpolation at *new* values.
Suppose you have 100 observations from some distribution. And let’s suppose
that you’re particularly interested in what’s happening around the middle.
You can mix pandas’ `reindex` and `interpolate` methods to interpolate
at the new values.

In [ ]:
ser = pd.Series(np.sort(np.random.uniform(size=100)))
# interpolate at new_index
new_index = ser.index.union(pd.Index([49.25, 49.5, 49.75, 50.25, 50.5, 50.75]))
interp_s = ser.reindex(new_index).interpolate(method="pchip")
interp_s[49:51]


<a id='working-with-missing-data-interp-limits'></a>

### Interpolation limits

Like other pandas fill methods, `interpolate()` accepts a `limit` keyword
argument. Use this argument to limit the number of consecutive `NaN` values
filled since the last valid observation:

In [ ]:
ser = pd.Series([np.nan, np.nan, 5, np.nan, np.nan, np.nan, 13, np.nan, np.nan])
ser

In [ ]:
# fill all consecutive values in a forward direction
ser.interpolate()

In [ ]:
# fill one consecutive value in a forward direction
ser.interpolate(limit=1)

By default, `NaN` values are filled in a `forward` direction. Use
`limit_direction` parameter to fill `backward` or from `both` directions.

In [ ]:
# fill one consecutive value backwards
ser.interpolate(limit=1, limit_direction="backward")

In [ ]:
# fill one consecutive value in both directions
ser.interpolate(limit=1, limit_direction="both")

In [ ]:
# fill all consecutive values in both directions
ser.interpolate(limit_direction="both")

By default, `NaN` values are filled whether they are inside (surrounded by)
existing valid values, or outside existing valid values. The `limit_area`
parameter restricts filling to either inside or outside values.

In [ ]:
# fill one consecutive inside value in both directions
ser.interpolate(limit_direction="both", limit_area="inside", limit=1)

In [ ]:
# fill all consecutive outside values backward
ser.interpolate(limit_direction="backward", limit_area="outside")

In [ ]:
# fill all consecutive outside values in both directions
ser.interpolate(limit_direction="both", limit_area="outside")


<a id='working-with-missing-data-replace'></a>

## Replacing generic values

Often times we want to replace arbitrary values with other values.

`replace()` in Series and `replace()` in DataFrame provides an efficient yet
flexible way to perform such replacements.

For a Series, you can replace a single value or a list of values by another
value:

In [ ]:
ser = pd.Series([0.0, 1.0, 2.0, 3.0, 4.0])

In [ ]:
ser.replace(0, 5)

You can replace a list of values by a list of other values:

In [ ]:
ser.replace([0, 1, 2, 3, 4], [4, 3, 2, 1, 0])

You can also specify a mapping dict:

In [ ]:
ser.replace({0: 10, 1: 100})

For a DataFrame, you can specify individual values by column:

In [ ]:
df = pd.DataFrame({"a": [0, 1, 2, 3, 4], "b": [5, 6, 7, 8, 9]})

In [ ]:
df.replace({"a": 0, "b": 5}, 100)

Instead of replacing with specified values, you can treat all given values as
missing and interpolate over them:

In [ ]:
ser.replace([1, 2, 3], method="pad")


<a id='working-with-missing-data-replace-expression'></a>

## String/regular expression replacement

>**Note**
>
>Python strings prefixed with the `r` character such as `r'hello world'`
are so-called “raw” strings. They have different semantics regarding
backslashes than strings without this prefix. Backslashes in raw strings
will be interpreted as an escaped backslash, e.g., `r'\' == '\\'`. You
should [read about them](https://docs.python.org/3/reference/lexical_analysis.html#string-literals)
if this is unclear.

Replace the ‘.’ with `NaN` (str -> str):

In [ ]:
d = {"a": list(range(4)), "b": list("ab.."), "c": ["a", "b", np.nan, "d"]}
df = pd.DataFrame(d)
df.replace(".", np.nan)

Now do it with a regular expression that removes surrounding whitespace
(regex -> regex):

In [ ]:
df.replace(r"\s*\.\s*", np.nan, regex=True)

Replace a few different values (list -> list):

In [ ]:
df.replace(["a", "."], ["b", np.nan])

list of regex -> list of regex:

In [ ]:
df.replace([r"\.", r"(a)"], ["dot", r"\1stuff"], regex=True)

Only search in column `'b'` (dict -> dict):

In [ ]:
df.replace({"b": "."}, {"b": np.nan})

Same as the previous example, but use a regular expression for
searching instead (dict of regex -> dict):

In [ ]:
df.replace({"b": r"\s*\.\s*"}, {"b": np.nan}, regex=True)

You can pass nested dictionaries of regular expressions that use `regex=True`:

In [ ]:
df.replace({"b": {"b": r""}}, regex=True)

Alternatively, you can pass the nested dictionary like so:

In [ ]:
df.replace(regex={"b": {r"\s*\.\s*": np.nan}})

You can also use the group of a regular expression match when replacing (dict
of regex -> dict of regex), this works for lists as well.

In [ ]:
df.replace({"b": r"\s*(\.)\s*"}, {"b": r"\1ty"}, regex=True)

You can pass a list of regular expressions, of which those that match
will be replaced with a scalar (list of regex -> regex).

In [ ]:
df.replace([r"\s*\.\s*", r"a|b"], np.nan, regex=True)

All of the regular expression examples can also be passed with the
`to_replace` argument as the `regex` argument. In this case the `value`
argument must be passed explicitly by name or `regex` must be a nested
dictionary. The previous example, in this case, would then be:

In [ ]:
df.replace(regex=[r"\s*\.\s*", r"a|b"], value=np.nan)

This can be convenient if you do not want to pass `regex=True` every time you
want to use a regular expression.

>**Note**
>
>Anywhere in the above `replace` examples that you see a regular expression
a compiled regular expression is valid as well.

## Numeric replacement

`replace()` is similar to `fillna()`.

In [ ]:
df = pd.DataFrame(np.random.randn(10, 2))
df[np.random.rand(df.shape[0]) > 0.5] = 1.5
df.replace(1.5, np.nan)

Replacing more than one value is possible by passing a list.

In [ ]:
df00 = df.iloc[0, 0]
df.replace([1.6, df00], [np.nan, "a"])

In [ ]:
df[1].dtype

You can also operate on the DataFrame in place:

In [ ]:
df.replace(1.5, np.nan, inplace=True)

### Missing data casting rules and indexing

While pandas supports storing arrays of integer and boolean type, these types
are not capable of storing missing data. Until we can switch to using a native
NA type in NumPy, we’ve established some “casting rules”. When a reindexing
operation introduces missing data, the Series will be cast according to the
rules introduced in the table below.

|data type|Cast to|
|:--------------------------------------:|:--------------------------------------:|
|integer|float|
|boolean|object|
|float|no cast|
|object|no cast|
For example:

In [ ]:
s = pd.Series(np.random.randn(5), index=[0, 2, 4, 6, 7])
s > 0

In [ ]:
(s > 0).dtype

In [ ]:
crit = (s > 0).reindex(list(range(8)))
crit

In [ ]:
crit.dtype

Ordinarily NumPy will complain if you try to use an object array (even if it
contains boolean values) instead of a boolean array to get or set values from
an ndarray (e.g. selecting values based on some criteria). If a boolean vector
contains NAs, an exception will be generated:

``` Python 

reindexed = s.reindex(list(range(8))).fillna(0)
reindexed[crit]
ValueError: Cannot mask with non-boolean array containing NA / NaN values

```


However, these can be filled in using `fillna()` and it will work fine:

In [ ]:
reindexed[crit.fillna(False)]

In [ ]:
reindexed[crit.fillna(True)]

pandas provides a nullable integer dtype, but you must explicitly request it
when creating the series or column. Notice that we use a capital “I” in
the `dtype="Int64"`.

In [ ]:
s = pd.Series([0, 1, np.nan, 3, 4], dtype="Int64")
s

See [Nullable integer data type](31_Nullable_integer_na.ipynb#integer-na) for more.


<a id='working-with-missing-data-na'></a>

## Experimental `NA` scalar to denote missing values

warning:

> Experimental: the behaviour of `pd.NA` can still change without warning.


New in version 1.0.0.

Starting from pandas 1.0, an experimental `pd.NA` value (singleton) is
available to represent scalar missing values. At this moment, it is used in
the nullable integer, boolean and
dedicated string data types as the missing value indicator.

The goal of `pd.NA` is provide a “missing” indicator that can be used
consistently across data types (instead of `np.nan`, `None` or `pd.NaT`
depending on the data type).

For example, when having missing values in a Series with the nullable integer
dtype, it will use `pd.NA`:

In [ ]:
s = pd.Series([1, 2, None], dtype="Int64")
s

In [ ]:
s[2]

In [ ]:
s[2] is pd.NA

Currently, pandas does not yet use those data types by default (when creating
a DataFrame or Series, or when reading in data), so you need to specify
the dtype explicitly.  An easy way to convert to those dtypes is explained
here.

### Propagation in arithmetic and comparison operations

In general, missing values *propagate* in operations involving `pd.NA`. When
one of the operands is unknown, the outcome of the operation is also unknown.

For example, `pd.NA` propagates in arithmetic operations, similarly to
`np.nan`:

In [ ]:
pd.NA + 1

In [ ]:
"a" * pd.NA

There are a few special cases when the result is known, even when one of the
operands is `NA`.

In [ ]:
pd.NA ** 0

In [ ]:
1 ** pd.NA

In equality and comparison operations, `pd.NA` also propagates. This deviates
from the behaviour of `np.nan`, where comparisons with `np.nan` always
return `False`.

In [ ]:
pd.NA == 1

In [ ]:
pd.NA == pd.NA

In [ ]:
pd.NA < 2.5

To check if a value is equal to `pd.NA`, the `isna()` function can be
used:

In [ ]:
pd.isna(pd.NA)

An exception on this basic propagation rule are *reductions* (such as the
mean or the minimum), where pandas defaults to skipping missing values. See
above for more.

### Logical operations

For logical operations, `pd.NA` follows the rules of the
[three-valued logic](https://en.wikipedia.org/wiki/Three-valued_logic) (or
*Kleene logic*, similarly to R, SQL and Julia). This logic means to only
propagate missing values when it is logically required.

For example, for the logical “or” operation (`|`), if one of the operands
is `True`, we already know the result will be `True`, regardless of the
other value (so regardless the missing value would be `True` or `False`).
In this case, `pd.NA` does not propagate:

In [ ]:
True | False

In [ ]:
True | pd.NA

In [ ]:
pd.NA | True

On the other hand, if one of the operands is `False`, the result depends
on the value of the other operand. Therefore, in this case `pd.NA`
propagates:

In [ ]:
False | True

In [ ]:
False | False

In [ ]:
False | pd.NA

The behaviour of the logical “and” operation (`&`) can be derived using
similar logic (where now `pd.NA` will not propagate if one of the operands
is already `False`):

In [ ]:
False & True

In [ ]:
False & False

In [ ]:
False & pd.NA

In [ ]:
True & True

In [ ]:
True & False

In [ ]:
True & pd.NA

### `NA` in a boolean context

Since the actual value of an NA is unknown, it is ambiguous to convert NA
to a boolean value. The following raises an error:


``` Python

bool(pd.NA)
TypeError: boolean value of NA is ambiguous
```

This also means that `pd.NA` cannot be used in a context where it is
evaluated to a boolean, such as `if condition: ...` where `condition` can
potentially be `pd.NA`. In such cases, `isna()` can be used to check
for `pd.NA` or `condition` being `pd.NA` can be avoided, for example by
filling missing values beforehand.

A similar situation occurs when using Series or DataFrame objects in `if`
statements, see [Using if/truth statements with pandas](43_FAQ_gotchas.ipynb#gotchas-truth).

### NumPy ufuncs

`pandas.NA` implements NumPy’s `__array_ufunc__` protocol. Most ufuncs
work with `NA`, and generally return `NA`:

In [ ]:
np.log(pd.NA)

In [ ]:
np.add(pd.NA, 1)

warning:

> Currently, ufuncs involving an ndarray and `NA` will return an
object-dtype filled with NA values.

In [ ]:
a = np.array([1, 2, 3])
np.greater(a, pd.NA)

The return type here may change to return a different array type
in the future.


See [DataFrame interoperability with NumPy functions](20_dsintro.ipynb#dsintro-numpy-interop) for more on ufuncs.


<a id='working-with-missing-data-na-conversion'></a>

### Conversion

If you have a DataFrame or Series using traditional types that have missing data
represented using `np.nan`, there are convenience methods
`convert_dtypes()` in Series and `convert_dtypes()`
in DataFrame that can convert data to use the newer dtypes for integers, strings and
booleans listed here. This is especially helpful after reading
in data sets when letting the readers such as `read_csv()` and `read_excel()`
infer default dtypes.

In this example, while the dtypes of all columns are changed, we show the results for
the first 10 columns.

In [ ]:
bb = pd.read_csv("../data/baseball.csv", index_col="id")
bb[bb.columns[:10]].dtypes

In [ ]:
bbn = bb.convert_dtypes()
bbn[bbn.columns[:10]].dtypes